In [11]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('DeepSearch'))))
from connector import es_connector
import json

In [29]:
#es.deleteIndex('textindex2')

In [31]:
es = es_connector.ES()

In [32]:
print(es.allIndex())

yellow open klue-mrc-dev                    EKq-LljHQmyceeV4R9LVRw 5 1  5076   0 19.6mb 19.6mb
green  open .kibana_-700776044_deepsearch_1 87d-qHhWRlqV1KivHOsUxg 1 0     4   1 15.6kb 15.6kb
green  open .opendistro_security            VFXXzkbeQGqDMR7T2JDSSg 1 0     9   4 59.5kb 59.5kb
green  open .kibana_1                       lvca5kdxTb2Y1LVm1NCPZg 1 0    67 191 71.5kb 71.5kb
yellow open klue-mrc-train                  IYNT_vsOTNyiHsaJ8thgEg 5 1 12174   0 49.6mb 49.6mb



In [33]:

with open('settings.json','r') as f:
    settings = json.load(f)
    
with open('mappings.json','r') as f:
    mappings = json.load(f)
    
es.createIndex(index='news-naver',settings=settings, mappings=mappings)

In [60]:
from utils import Get_Article_Body, ariticle_reply, get_article_df
import datetime
from selenium.webdriver import Chrome
import json
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('DeepSearch'))))

today = datetime.datetime.now()
yesterday = today + datetime.timedelta(days=-1)


# 검색 키워드 입력
search_keywords = ['삼성생명']

# 기간 설정
years = [yesterday.strftime("%Y-%m-%d")[:4]]
months = [datetime.datetime.now().strftime("%Y-%m-%d")[5:7]]
all_days = [yesterday.strftime("%Y-%m-%d")[-2:]]

driver = Chrome()
article_url_df = get_article_df(years, months, all_days, search_keywords, driver)
urls = article_url_df['url']
#기사 내용 수집
article_dicts = []
reply_dfs = []


In [ ]:
for url in urls:
    try:
        article_dict = Get_Article_Body(url,driver)
        article_dict['search_keywords'] = search_keywords[0]
        es.dataInsert(index='news-naver', data=article_dict)
        #reply_df = ariticle_reply()
        #reply_dfs.append(reply_df)
    except:
        continue